In [3]:
import pandas as pd 

dataframe = pd.read_csv("skin_segmentation.csv")
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245057 entries, 0 to 245056
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   V1      245057 non-null  int64
 1   V2      245057 non-null  int64
 2   V3      245057 non-null  int64
 3   Class   245057 non-null  int64
dtypes: int64(4)
memory usage: 7.5 MB


In [4]:
from sklearn.model_selection import train_test_split
X = dataframe.drop(columns=['Class'])
y = dataframe['Class']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [5]:
# Launch tpot training 
from tpot import TPOTClassifier
clf_tpot = TPOTClassifier(verbosity=2, max_time_mins=30 )
clf_tpot.fit(X_train, y_train)

/home/datagenius/anaconda3/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")



32.63 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.9000000000000001, min_samples_leaf=2, min_samples_split=3, n_estimators=100)


TPOTClassifier(max_time_mins=30, verbosity=2)

In [6]:
import autosklearn.classification
clf_autosklearn = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=1800)
clf_autosklearn.fit(X_train, y_train)

AutoSklearnClassifier(per_run_time_limit=180, time_left_for_this_task=1800)

In [7]:
from oboe import AutoLearner, error
import numpy as np 
xx_train = np.array(X_train)
yy_train = np.array(y_train)
method = 'Oboe' # 'Oboe' or 'TensorOboe'
problem_type = 'classification'
clf_oboe = AutoLearner(p_type=problem_type, runtime_limit=30, method=method, verbose=False)
clf_oboe.fit(xx_train, yy_train)

0.0015302609094850672


/home/datagenius/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/datagenius/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/datagenius/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/datagenius/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/datagenius/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Lib

{'ranks': [8, 9, 10, 11, 12],
 'runtime_limits': [1, 2, 4, 8, 16],
 'validation_loss': [0.5, 0.0, 0.0, 0.0, 0.0, 0.0],
 'filled_new_row': [array([[ 5.02945617e-02,  4.20208796e-02,  4.32315083e-01,
           7.09156201e-01,  7.24933527e-01,  4.32189654e-02,
           3.42212713e-02,  4.31254601e-01,  7.04771656e-01,
           7.20861705e-01,  6.73187763e-02,  7.02844144e-02,
           6.66608722e-02,  8.12827137e-02,  1.06258147e-01,
           1.37602105e-01,  2.46303973e-01,  4.80211194e-01,
           4.80946872e-01,  5.22353729e-01,  7.23876429e-02,
           6.75767129e-02,  6.73187763e-02,  6.73187763e-02,
           3.07293991e-02,  1.88129695e-02,  1.93847038e-02,
           1.27944835e-02,  2.26897055e-02,  9.17740099e-03,
           3.76030537e-02,  2.30988635e-02,  6.83493660e-02,
           5.05589661e-02,  1.34181119e-01,  1.12126646e-01,
           2.43016660e-01,  2.27291060e-01,  4.75767936e-01,
           4.58712782e-01,  5.19053247e-01,  5.15277802e-01,
         

In [8]:
from sklearn.metrics import accuracy_score
print("tpot score:\t\t ", clf_tpot.score(X_test, y_test))
print("autosklern score:\t\t ", clf_autosklearn.score(X_test, y_test))

xx_test = np.array(X_test)
yy_test = np.array(y_test)
y_predicted = clf_oboe.predict(xx_test)[0]
print("oboe score:\t\t ", accuracy_score(y_test, y_predicted) )

tpot score:		  0.9996327430017139
autosklern score:		  0.9996939525014282
oboe score:		  0.9892679343834163


In [9]:
tpot_pred    = clf_tpot.predict(X_test)
sklearn_pred = clf_autosklearn.predict(X_test)
oboe_pred    = clf_oboe.predict(X_test)[0]

In [13]:
prob = pd.DataFrame(list(zip(sklearn_pred, oboe_pred, tpot_pred)), columns=[ "AutoSklearn_Class", "Oboe_Class", "TPOT_Class"])
def function(x) :
    if [x['AutoSklearn_Class'],x['Oboe_Class'], x['TPOT_Class']].count(1)>=2 :
        return 1
    else   :
        return 2
    # else :
    #     int(x['AutoSklearn_Class'])
prob['ensemble'] = prob.apply(function, axis=1)

In [14]:
print(accuracy_score(y_test.astype(int), prob['ensemble'] ))

0.9995919366685709
